In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib

In [2]:
from datetime import datetime, timedelta

In [3]:
%matplotlib inline

In [4]:
input_directory = './input/'
data = gpd.read_file( input_directory + 'Contiguous_US.geojson')

In [5]:
global number_counties 
number_counties = data.shape[0] #3103

In [6]:
data.sort_values(by='GEOID', inplace=True)

### Load COVID data and apply smoothing (moveing average)

In [7]:
jh_covid_df = pd.read_csv(input_directory + 'time_series_covid19_confirmed_US.csv')
jh_covid_df.shape

(3340, 358)

In [8]:
jh_covid_df.dropna(axis=0, how='any', inplace=True)

In [9]:
jh_covid_df.shape

(3324, 358)

In [10]:
jh_covid_df['FIPS'] = jh_covid_df['FIPS'].astype('int64')

In [11]:
jh_covid_df.drop(columns=['UID', 'iso2', 'iso3', 'code3','Country_Region', 'Lat', 'Long_'], inplace=True)

In [12]:
jh_covid_df

,FIPS,Admin2,Province_State,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21
0,1001,Autauga,Alabama,"Autauga, Alabama, US",0,0,0,0,0,0,...,3942,3990,3999,4029,4065,4105,4164,4190,4239,4268
1,1003,Baldwin,Alabama,"Baldwin, Alabama, US",0,0,0,0,0,0,...,12521,12666,12708,12825,12962,13172,13392,13601,13823,13955
2,1005,Barbour,Alabama,"Barbour, Alabama, US",0,0,0,0,0,0,...,1390,1396,1398,1406,1417,1462,1492,1514,1517,1528
3,1007,Bibb,Alabama,"Bibb, Alabama, US",0,0,0,0,0,0,...,1711,1725,1739,1746,1762,1792,1817,1834,1854,1863
4,1009,Blount,Alabama,"Blount, Alabama, US",0,0,0,0,0,0,...,4405,4441,4446,4465,4483,4535,4584,4641,4693,4729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,56039,Teton,Wyoming,"Teton, Wyoming, US",0,0,0,0,0,0,...,2039,2039,2080,2080,2108,2112,2121,2138,2138,2145
3336,56041,Uinta,Wyoming,"Uinta, Wyoming, US",0,0,0,0,0,0,...,1475,1475,1495,1497,1507,1521,1540,1558,1558,1569
3337,90056,Unassigned,Wyoming,"Unassigned, Wyoming, US",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3338,56043,Washakie,Wyoming,"Washakie, Wyoming, US",0,0,0,0,0,0,...,726,726,751,747,763,764,768,781,781,782


In [13]:
covid_df = jh_covid_df.copy()

In [14]:
from datetime import datetime, timedelta

#Important: check to see the column index is adherent to the imported df

first_date = datetime.strptime(covid_df.columns[4], '%m/%d/%y').date()

last_date = datetime.strptime(covid_df.columns[-1], '%m/%d/%y').date()

In [15]:
current_date= last_date

previous_date = last_date - timedelta (days=1)


while current_date > first_date:

    #For unix, replace # with - in the time format

    current_col = current_date.strftime('%#m/%#d/%y')

    previous_col = previous_date.strftime('%#m/%#d/%y')

    covid_df[previous_col] = np.where(covid_df[previous_col] > covid_df[current_col], covid_df[current_col], covid_df[previous_col])

    current_date = current_date - timedelta(days=1)

    previous_date = previous_date - timedelta(days=1)

In [16]:
# convert FIPS values in from int to str because FIPS values in cases and death DFs are integers in the
# original files
def parse(x):
    try:
        return str(x)
    except ValueError:
        return np.nan

In [17]:
covid_smooth = covid_df.copy()
covid_smooth['FIPS'] = covid_smooth['FIPS'].apply(parse)
covid_smooth.set_index('FIPS', inplace=True)

In [18]:
from datetime import datetime as dt
keys = covid_df.columns[4:]
values = pd.to_datetime(covid_df.columns[4:]).date
new_cols = {keys[i]:values[i] for i in range(len(keys))}

In [19]:
covid_smooth.rename(columns=new_cols, inplace=True)
covid_smooth = covid_smooth.T

In [20]:
covid_smooth

FIPS,1001,1003,1005,1007,1009,1011,1013,1015,1017,1019,...,56029,56031,56033,56035,56037,56039,56041,90056,56043,56045
Admin2,Autauga,Baldwin,Barbour,Bibb,Blount,Bullock,Butler,Calhoun,Chambers,Cherokee,...,Park,Platte,Sheridan,Sublette,Sweetwater,Teton,Uinta,Unassigned,Washakie,Weston
Province_State,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,...,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming
Combined_Key,"Autauga, Alabama, US","Baldwin, Alabama, US","Barbour, Alabama, US","Bibb, Alabama, US","Blount, Alabama, US","Bullock, Alabama, US","Butler, Alabama, US","Calhoun, Alabama, US","Chambers, Alabama, US","Cherokee, Alabama, US",...,"Park, Wyoming, US","Platte, Wyoming, US","Sheridan, Wyoming, US","Sublette, Wyoming, US","Sweetwater, Wyoming, US","Teton, Wyoming, US","Uinta, Wyoming, US","Unassigned, Wyoming, US","Washakie, Wyoming, US","Weston, Wyoming, US"
2020-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-29,4105,13172,1462,1792,4535,834,1482,9345,2275,1393,...,1971,425,2507,590,2929,2112,1521,0,764,471
2020-12-30,4164,13392,1492,1817,4584,846,1493,9428,2310,1403,...,1985,434,2521,592,2943,2121,1540,0,768,475
2020-12-31,4190,13601,1514,1834,4641,859,1508,9494,2341,1414,...,2021,440,2543,597,2966,2138,1558,0,781,476
2021-01-01,4239,13823,1517,1854,4693,888,1522,9584,2366,1429,...,2021,440,2543,597,2966,2138,1558,0,781,476


In [21]:
#covid_smooth.drop('0', axis=1, inplace=True)

In [22]:
covid_smooth.iloc[3:,:] = covid_smooth.iloc[3:,:].rolling(7,min_periods=1 ).mean()

In [23]:
covid_smooth.index = covid_smooth.index.map(str)
covid_smooth_transpose = covid_smooth.T

In [24]:
covid_df = covid_smooth_transpose.copy()
covid_df.reset_index(inplace=True)

In [25]:
# convert FIPS values back to int
# original files
def parse_int(x):
    try:
        return int(x)
    except ValueError:
        return np.nan

covid_df['FIPS'] = covid_df['FIPS'].apply(parse_int)

In [26]:
cols = covid_df.iloc[:,4:].columns
covid_df[cols] = covid_df[cols].apply(pd.to_numeric, errors='coerce')

In [27]:
covid_df.head()

,FIPS,Admin2,Province_State,Combined_Key,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02
0,1001,Autauga,Alabama,"Autauga, Alabama, US",0.0,0.0,0.0,0.0,0.0,0.0,...,3791.142857,3840.142857,3883.142857,3924.285714,3965.000000,4002.714286,4042.000000,4077.428571,4113.000000,4151.428571
1,1003,Baldwin,Alabama,"Baldwin, Alabama, US",0.0,0.0,0.0,0.0,0.0,0.0,...,12007.714286,12166.285714,12307.142857,12449.714286,12594.000000,12739.285714,12892.285714,13046.571429,13211.857143,13390.000000
2,1005,Barbour,Alabama,"Barbour, Alabama, US",0.0,0.0,0.0,0.0,0.0,0.0,...,1350.857143,1362.000000,1371.714286,1381.714286,1393.285714,1407.428571,1423.000000,1440.714286,1458.000000,1476.571429
3,1007,Bibb,Alabama,"Bibb, Alabama, US",0.0,0.0,0.0,0.0,0.0,0.0,...,1639.000000,1660.142857,1679.857143,1698.857143,1718.000000,1736.857143,1756.000000,1773.571429,1792.000000,1809.714286
4,1009,Blount,Alabama,"Blount, Alabama, US",0.0,0.0,0.0,0.0,0.0,0.0,...,4263.714286,4309.857143,4346.285714,4381.571429,4417.142857,4448.857143,4479.857143,4513.571429,4549.571429,4590.000000


### Load Facebook Movement Range Data

In [31]:
fb_mobility = pd.read_csv(input_directory + 'movement-range-2020-12-13.txt', sep="\t", dtype={'polygon_id':str})

In [32]:
fb_mobility.head()

,ds,country,polygon_source,polygon_id,polygon_name,all_day_bing_tiles_visited_relative_change,all_day_ratio_single_tile_users,baseline_name,baseline_type
0,2020-03-01,AGO,GADM,AGO.10.10_1,Lubango,-0.02992,0.18751,full_february,DAY_OF_WEEK
1,2020-03-02,AGO,GADM,AGO.10.10_1,Lubango,0.06746,0.10521,full_february,DAY_OF_WEEK
2,2020-03-03,AGO,GADM,AGO.10.10_1,Lubango,0.05873,0.11397,full_february,DAY_OF_WEEK
3,2020-03-04,AGO,GADM,AGO.10.10_1,Lubango,0.01288,0.10492,full_february,DAY_OF_WEEK
4,2020-03-05,AGO,GADM,AGO.10.10_1,Lubango,0.02753,0.11056,full_february,DAY_OF_WEEK


In [33]:
fb_mobility_us = fb_mobility[fb_mobility['country']=='USA']

In [34]:
# Number of unique counties for which we have at least one day of data
len(fb_mobility_us['polygon_id'].unique())

2694

In [35]:
# counting Counties in the contiguous US for which there is no data in FB mobility
contiguous_fips = set(data['GEOID']) # number of unique fips: 3103
mobility_fips = set(fb_mobility_us['polygon_id']) # number of unique fips: 2694
i = 0
missing_fips = []
for fips in contiguous_fips:
    if (fips in mobility_fips):
        i+=1
    else:
        missing_fips.append(fips)

In [36]:
# number of Counties in the contiguous US for which there is no data in FB mobility
len(missing_fips)

512

In [37]:
# Create a new dataframe as transpose of the above, with days as columns and counties as rows

relative_df = pd.DataFrame(columns=fb_mobility_us['ds'].unique(), index=data['GEOID'].unique(), dtype=float)
ratio_df = pd.DataFrame(columns=fb_mobility_us['ds'].unique(), index=data['GEOID'].unique(), dtype=float)

In [38]:
fb_mobility_us

,ds,country,polygon_source,polygon_id,polygon_name,all_day_bing_tiles_visited_relative_change,all_day_ratio_single_tile_users,baseline_name,baseline_type
3793616,2020-03-01,USA,FIPS,01001,Autauga,0.04845,0.18910,full_february_except_presidents_day,DAY_OF_WEEK
3793617,2020-03-02,USA,FIPS,01001,Autauga,0.03275,0.15003,full_february_except_presidents_day,DAY_OF_WEEK
3793618,2020-03-03,USA,FIPS,01001,Autauga,0.05122,0.14391,full_february_except_presidents_day,DAY_OF_WEEK
3793619,2020-03-04,USA,FIPS,01001,Autauga,-0.04741,0.16058,full_february_except_presidents_day,DAY_OF_WEEK
3793620,2020-03-05,USA,FIPS,01001,Autauga,0.06285,0.15298,full_february_except_presidents_day,DAY_OF_WEEK
...,...,...,...,...,...,...,...,...,...
4519059,2020-12-09,USA,FIPS,78030,St. Thomas,-0.13617,0.15931,full_february_except_presidents_day,DAY_OF_WEEK
4519060,2020-12-10,USA,FIPS,78030,St. Thomas,-0.12228,0.16182,full_february_except_presidents_day,DAY_OF_WEEK
4519061,2020-12-11,USA,FIPS,78030,St. Thomas,-0.08423,0.15636,full_february_except_presidents_day,DAY_OF_WEEK
4519062,2020-12-12,USA,FIPS,78030,St. Thomas,-0.05475,0.17079,full_february_except_presidents_day,DAY_OF_WEEK


In [39]:
idx_contiguous = fb_mobility_us.index[fb_mobility_us['polygon_id'].isin(contiguous_fips)]

In [40]:
fb_mobility_contiguous = fb_mobility_us.loc[idx_contiguous]

In [41]:
import time
start = time.time()

for index, row in fb_mobility_contiguous.iterrows():
    relative_df.loc[row['polygon_id']][row['ds']] = row['all_day_bing_tiles_visited_relative_change']
    ratio_df.loc[row['polygon_id']][row['ds']] = row['all_day_ratio_single_tile_users']
    

end = time.time()
print(end - start)

139.30900645256042


In [42]:
relative_df.head()

,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,...,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13
01001,0.04845,0.03275,0.05122,-0.04741,0.06285,0.03601,0.08057,0.05780,0.09284,-0.01149,...,-0.10508,-0.06957,-0.09401,-0.02695,-0.06830,-0.05107,-0.01889,-0.09506,-0.09946,-0.11075
01003,0.14298,0.06573,0.09793,-0.04300,0.03726,0.02851,0.05436,0.12925,0.08460,0.09411,...,-0.07103,-0.07174,-0.05708,-0.01583,-0.02900,-0.02509,0.02164,-0.03777,-0.05612,-0.01147
01005,0.19482,0.04019,-0.04098,-0.09939,-0.07579,0.01357,0.07628,0.15712,0.07074,-0.01814,...,-0.11686,-0.04082,-0.06815,-0.02797,-0.11775,-0.05084,0.03254,-0.08609,-0.08835,-0.05707
01007,0.07842,0.05036,0.14394,-0.06774,0.00058,0.08248,0.09590,0.09463,0.12007,0.05285,...,-0.06317,0.00585,-0.02071,0.05409,0.02844,-0.06518,-0.03114,-0.05754,-0.09495,-0.05755
01009,0.03646,0.00816,0.08475,-0.02877,-0.02288,0.03908,0.05726,-0.03108,0.10135,0.00426,...,-0.11363,-0.02090,-0.12098,-0.01961,-0.05861,-0.05886,-0.01828,-0.03026,-0.13229,-0.11809


In [43]:
print(relative_df.shape , ratio_df.shape)

(3103, 288) (3103, 288)


In [55]:
#relative_df.to_csv('./output/FB-relative-contiguous.csv')
#ratio_df.to_csv('./output/FB-ratio-contiguous.csv')

#### Impute FB mobility dataframes
The two dataframes above have a lot of Nan values which should be imputed by state average

In [44]:
temp_ratio_df = data[['GEOID', 'STATEFP']].merge(ratio_df, left_on='GEOID', right_index=True, how='left')

In [45]:
for col in temp_ratio_df.iloc[:,2:].columns:
    temp_ratio_df[col] = temp_ratio_df.groupby('STATEFP')[col].transform(lambda x: x.fillna(x.mean()))

In [46]:
temp_ratio_df.isna().sum().sum()

0

In [47]:
temp_relative_df = data[['GEOID', 'STATEFP']].merge(relative_df, left_on='GEOID', right_index=True, how='left')

In [48]:
for col in temp_relative_df.iloc[:,2:].columns:
    temp_relative_df[col] = temp_relative_df.groupby('STATEFP')[col].transform(lambda x: x.fillna(x.mean()))

In [49]:
temp_relative_df.isna().sum().sum()

0

In [50]:
relative_df_smooth = temp_relative_df.copy()
ratio_df_smooth = temp_ratio_df.copy()

relative_df_smooth.iloc[:,2:] = relative_df_smooth.iloc[:,2:].rolling(7,min_periods=1, axis=1).mean()
ratio_df_smooth.iloc[:,2:] = ratio_df_smooth.iloc[:,2:].rolling(7,min_periods=1, axis=1).mean()

In [49]:
relative_df_smooth.iloc[:,2:].columns

Index(['2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10',
       ...
       '2020-12-04', '2020-12-05', '2020-12-06', '2020-12-07', '2020-12-08',
       '2020-12-09', '2020-12-10', '2020-12-11', '2020-12-12', '2020-12-13'],
      dtype='object', length=288)

In [51]:
relative_df_smooth.shape

(3103, 290)

## Add Social Proximity to Cases

In [52]:
SCI_df = pd.read_csv(input_directory + 'SCI_matrix.csv', dtype={'Unnamed: 0':str})

In [53]:
SCI_df.set_index('Unnamed: 0', inplace=True)

In [54]:
# Create normalized SCI. It is calculated by dividing all the columns of the sci_matrix by the sum of the rpw
# This would give us the second term in social proximity formula above

sci_matrix_normal = SCI_df.div(SCI_df.sum(axis=1), axis=0)

In [55]:
# set diagonal to zero
sci_matrix_normal.values[[np.arange(sci_matrix_normal.shape[0])]*2] = 0

C:\Users\BehzadVahediTorghabe\anaconda3\envs\tf-2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


**The matrix above is created for the entire US, but we are using contiguous US data here, therefore some rows and columns should be removed**

In [58]:
to_drop=[]

for index in sci_matrix_normal.index:
    if not index in contiguous_fips:
        to_drop.append(index)

In [59]:
sci_matrix_normal.drop(to_drop, inplace=True)

In [60]:
sci_matrix_normal.drop(to_drop, axis=1, inplace=True)

In [61]:
sci_matrix_normal.shape

(3103, 3103)

In [62]:
data['POP_DENSITY'] = data['POPULATION'] / data['ALAND']

## Add SafeGraph mobility features

Updated based on forecast hub dates

In [63]:
safegraph_mobility = pd.read_csv(input_directory + 'weekly_df_updated_dates_feb20.csv', dtype={'county_fips':str})


In [64]:
safegraph_mobility.head()

,county_fips,pct_completely_home_device_count_current,pct_full_time_work_behavior_devices_current,pct_part_time_work_behavior_devices_current,pct_delivery_behavior_devices_current,distance_traveled_from_home_current,median_home_dwell_time_current,pct_completely_home_device_count_baselined,pct_full_time_work_behavior_devices_baselined,pct_part_time_work_behavior_devices_baselined,...,start_date,end_date,base_start,base_end,pct_completely_home_device_count_slope,pct_full_time_work_behavior_devices_slope,pct_part_time_work_behavior_devices_slope,pct_delivery_behavior_devices_slope,distance_traveled_from_home_slope,median_home_dwell_time_slope
0,01001,0.203983,0.091998,0.159007,0.035317,10975.011588,699.704348,1.139654,-0.410105,-1.241178,...,2020-02-23,2020-02-29,2020-02-02,2020-02-08,0.484738,-0.815895,-0.264356,0.563745,0.439380,-0.935252
1,01003,0.215158,0.067916,0.134442,0.026220,10019.199748,601.665701,0.560501,-1.921581,-1.998418,...,2020-02-23,2020-02-29,2020-02-02,2020-02-08,0.555678,-0.882163,-0.758854,-0.387523,0.787119,-0.810073
2,01005,0.216429,0.066053,0.129262,0.037767,13504.626966,511.994220,-1.429781,-0.130913,-0.247291,...,2020-02-23,2020-02-29,2020-02-02,2020-02-08,0.218528,-0.846283,-0.723246,0.771437,0.888783,-0.935333
3,01007,0.203530,0.085612,0.145061,0.036371,12751.994270,661.345262,1.533934,-0.980339,-1.044010,...,2020-02-23,2020-02-29,2020-02-02,2020-02-08,0.500807,-0.791998,-0.634945,0.906629,0.680636,-0.972836
4,01009,0.197803,0.092337,0.150117,0.032805,15077.498622,689.338117,0.942498,-0.566777,-0.664327,...,2020-02-23,2020-02-29,2020-02-02,2020-02-08,0.630965,-0.836106,-0.599603,-0.506209,-0.342803,-0.962212


In [66]:
safegraph_contiguous = safegraph_mobility[safegraph_mobility['county_fips'].isin(contiguous_fips)]

In [67]:
len(safegraph_contiguous['county_fips'].unique())

3103

In [70]:
temp_cols = safegraph_contiguous.drop(['start_date', 'end_date', 'base_start', 'base_end'], axis=1)
safegraph_metrics = temp_cols.columns

In [72]:
len(safegraph_metrics)

19

## Load Temperature Data

In [75]:
max_temp = pd.read_csv(input_directory + 'max_temp_df.csv', dtype={'GEOID':str})


In [76]:
max_temp.head()

,GEOID,2020-03-01,2020-03-08,2020-03-15,2020-03-22,2020-03-29,2020-04-05,2020-04-12,2020-04-19,2020-04-26,...,2020-09-27,2020-10-04,2020-10-11,2020-10-18,2020-10-25,2020-11-01,2020-11-08,2020-11-15,2020-11-22,2020-11-29
0,51145,12.180214,16.878896,9.318125,12.461617,30.335473,22.286410,23.086930,19.454080,19.339914,...,24.136514,19.962470,20.177032,18.615678,11.251407,16.715503,24.829473,21.916389,17.499279,15.459098
1,51515,13.576157,17.031094,8.328685,11.419327,29.483562,22.393398,20.317541,18.134025,19.137850,...,24.928335,19.692350,18.589684,17.186888,11.967191,18.244125,24.127651,19.377673,16.348371,14.187460
2,01001,20.790716,20.492847,27.073105,20.226255,26.568584,28.495850,27.087422,17.528677,23.276668,...,30.335370,24.420849,24.890266,26.712361,24.476239,23.282393,24.645829,24.185170,23.874073,16.572760
3,01003,20.439063,19.562366,28.134529,24.866176,28.662859,26.281934,25.748271,27.123152,25.782710,...,30.665151,25.005359,28.712712,27.420471,24.177378,24.014076,27.486124,27.680297,24.842339,21.015124
4,01005,20.301218,18.986057,27.630619,21.987266,29.729225,27.406309,27.496108,21.056312,23.675344,...,26.952850,23.315882,26.060817,26.908917,25.181664,23.477150,25.275973,25.693962,23.790892,19.480325


In [77]:
min_temp = pd.read_csv(input_directory + 'min_temp_df.csv', dtype={'GEOID':str})


In [78]:
min_temp.head()

,GEOID,2020-03-01,2020-03-08,2020-03-15,2020-03-22,2020-03-29,2020-04-05,2020-04-12,2020-04-19,2020-04-26,...,2020-09-27,2020-10-04,2020-10-11,2020-10-18,2020-10-25,2020-11-01,2020-11-08,2020-11-15,2020-11-22,2020-11-29
0,51145,-6.402456,-4.516921,5.407520,5.029067,12.329676,3.154822,6.637645,-0.775483,9.986511,...,16.717731,5.083667,16.238845,2.622027,9.698364,5.168125,8.116226,6.805375,8.261407,0.231648
1,51515,-5.618476,-4.398159,4.476370,5.370979,13.015199,2.877679,1.534509,-1.312804,9.846000,...,12.955646,4.350055,14.233291,1.310957,10.758430,3.284169,5.223021,4.157880,4.993098,-1.608654
2,01001,1.264055,3.415388,15.783860,10.035215,20.938789,15.422846,8.678049,12.427319,11.602209,...,19.697185,9.146586,20.900049,10.853019,18.129397,10.659677,17.675837,10.901502,8.914933,10.970921
3,01003,4.460209,8.636308,13.987997,14.923723,22.456325,16.458706,17.423446,18.408608,14.978750,...,20.913819,11.847546,21.902287,12.850911,18.065851,13.360227,19.120757,15.673550,12.214525,14.402062
4,01005,1.912538,3.455319,14.176521,13.188881,20.435224,15.403058,8.923684,15.136977,14.217469,...,19.865491,9.798039,22.293161,11.556852,18.471529,12.441852,18.313776,10.660652,12.199125,13.284400


## Helper functions

In [79]:
# return FCI-normal table for the input date
# set path_to_fci to where FCI matrices are stored
def get_normal_fci(date):
    path_to_fci = './output/2020/' + date.strftime('%Y-%m-%d').split('-')[1] + '/FCI_normal/' + date.strftime('%Y-%m-%d') + '-FCI-normal.csv'
    fci_norm = pd.read_csv(path_to_fci, dtype={'Unnamed: 0':str})
    fci_norm.set_index('Unnamed: 0', inplace=True)
    
    to_drop=[]

    for index in fci_norm.index:
        if not index in contiguous_fips:
            to_drop.append(index)
            
    fci_norm.drop(to_drop, inplace=True)
    fci_norm.drop(to_drop, axis=1, inplace=True)
    return fci_norm

In [80]:
# calculates weekly average FPC using the end date and the start date of the week
# the input to this fuction should be of type datetime
def weekly_mean_FPC(end_date, start_date):
    
    dates = [end_date]
    while end_date> start_date:
        end_date -= timedelta(days=1)
        dates.append(end_date)
    
    
    temp = data[['GEOID','FIPS', 'POPULATION']]
    
    
    for date in dates:
        # convert date to String
        date_str = date.strftime('%Y-%m-%d')
        
        temp = temp.merge(covid_df_contiguous[['FIPS', date_str]], on='FIPS', how='left')
        temp['inc_rate_' + date_str] = temp[date_str] / temp['POPULATION'] * 10000
        
        normal_fci = get_normal_fci(date)
        normal_fci = normal_fci.merge(temp, left_index= True, right_on='GEOID')
        
        normal_fci['fpc_'+ date_str] = np.dot(normal_fci.iloc[:,:number_counties], normal_fci['inc_rate_' + date_str])
        
    
    normal_fci['mean_fpc'] = normal_fci.iloc[:,-len(dates):].mean(axis=1)
        
    return normal_fci[['GEOID','mean_fpc']]    

In [81]:
# calculates weekly average SPC
# the input to this fuction should be of type datetime
def weekly_mean_SPC(end_date, start_date):
    
    dates = [end_date]
    while end_date> start_date:
        end_date -= timedelta(days=1)
        dates.append(end_date)
    
    
    temp = data[['GEOID','FIPS', 'POPULATION']]
    
    for date in dates:
        # convert date to String
        date_str = date.strftime('%Y-%m-%d')
        
        temp = temp.merge(covid_df_contiguous[['FIPS', date_str]], on='FIPS', how='left')
        temp['inc_rate_' + date_str] = temp[date_str] / temp['POPULATION'] * 10000
        
        
        normal_sci = sci_matrix_normal.merge(temp, left_index= True, right_on='GEOID')
        
        normal_sci['spc_'+ date_str] = np.dot(normal_sci.iloc[:,:number_counties], normal_sci['inc_rate_' + date_str])
        
    
    normal_sci['mean_spc'] = normal_sci.iloc[:,-len(dates):].mean(axis=1)
        
    return normal_sci[['GEOID','mean_spc']]

In [82]:
# the input to this fuction should be of type datetime.
# returns a subset of FB movement range dfs based on the given week
def weekly_fb_mobility(end_date, start_date, df):
    
    dates = [end_date]
    while end_date> start_date:
        end_date -= timedelta(days=1)
        dates.append(end_date)
    
    dates_str=[]
    for date in dates:
        # convert date to String
        dates_str.append(date.strftime('%Y-%m-%d'))
    
    return df[['GEOID', *dates_str]]

In [83]:
from scipy.stats import linregress

def linear_reg(week_df):
#     return_df = pd.DataFrame(columns = grouped_df.columns)
#     return_df = return_df.append(pd.Series(), ignore_index=True)
    
    x = np.arange(1,(week_df.shape[1]),1)
    x = (x - np.mean(x))/ np.std(x)
    
    slopes=[]
    
    for index, row in week_df.iloc[:,1:].iterrows():
        y = row
        y = (y - np.mean(y))/ np.std(y)
        slopes.append(linregress(x, y)[0])
        
    week_df.loc[:,'slope'] = slopes
    return week_df[['GEOID','slope']]

## Create a function to add data based on specific dates

For a given date, this function will add number of cases, incidence rates, SPC, and .. to data

- incidence rate data
- FB mobility data (ratio, relative)
- SPC (facebook SCI and incidence rates)
- SafeGraph mobility 
- FPC (FCI and incidence rate)

For each period, there is a 5 week difference between the actual date (t) and the start of the second lag. For example if `T: Oct 1 (Sep 24 to Oct 1)`, then `T-1: Sep 10 to Sep 24`, and `T-2: August 27 to Sep 10`.

Since the earliest day for which we have FB mobility data is March 1, the rearliest  (end) date for T will be April 5th.

In [84]:
data.shape

(3103, 79)

In [85]:
covid_df_contiguous = data[['FIPS','STATEFP','COUNTYFP','GEOID']].merge(covid_df, on='FIPS', how='left')

In [96]:
# Main function
# the input to the funtion is the end date for which we want to add data

from datetime import datetime, timedelta


def add_lagged_data(date):
    global data
    
    T_end = date
    T_start = T_end - timedelta(days=6)
    
    T_1_end = T_start - timedelta(days=1)
    T_1_start = T_1_end - timedelta(days=6)
    
    T_2_end = T_1_start - timedelta(days=1)
    T_2_start = T_2_end - timedelta(days=6)
    
    T_3_end = T_2_start - timedelta(days=1)
    T_3_start = T_3_end - timedelta(days=6)
    
    T_4_end = T_3_start - timedelta(days=1)
    T_4_start = T_4_end - timedelta(days=6)

    dates = [T_end.strftime('%Y-%m-%d'), T_start.strftime('%Y-%m-%d'),
             T_1_end.strftime('%Y-%m-%d'), T_1_start.strftime('%Y-%m-%d'), 
             T_2_end.strftime('%Y-%m-%d'), T_2_start.strftime('%Y-%m-%d'),
             T_3_end.strftime('%Y-%m-%d'), T_3_start.strftime('%Y-%m-%d'), 
             T_4_end.strftime('%Y-%m-%d'), T_4_start.strftime('%Y-%m-%d')]
    
    dates_non_str = [T_end, T_start,
             T_1_end, T_1_start, 
             T_2_end, T_2_start,
             T_3_end, T_3_start, 
             T_4_end, T_4_start]


    temp = data.copy()
    
    temp['date_end_period'] = T_end.strftime('%Y-%m-%d')
    temp['date_start_period'] = T_start.strftime('%Y-%m-%d')
    temp['date_end_lag'] = T_1_end.strftime('%Y-%m-%d')
    temp['date_start_lag'] = T_4_start.strftime('%Y-%m-%d')
    
    time_periods = ['T_end', 'T_start', 'T_1_end', 'T_1_start','T_2_end','T_2_start',
                    'T_3_end','T_3_start','T_4_end','T_4_start']
    i = 0
    for period in time_periods:
        
        
        temp = temp.merge(covid_df_contiguous[['GEOID',dates[i]]], on='GEOID', how='left')
        temp.rename(columns={dates[i]:'case_'+ period}, inplace=True)

        
        temp['inc_rate_' + period] = temp['case_'+ period] / temp['POPULATION'] * 10000
        
        temp = temp.merge(relative_df_smooth[['GEOID',dates[i]]], on='GEOID', how='left')
        temp.rename(columns={dates[i]:'relative_'+ period}, inplace=True) 


        temp = temp.merge(ratio_df_smooth[['GEOID',dates[i]]], on='GEOID', how='left')
        temp.rename(columns={dates[i]:'ratio_'+ period}, inplace=True)
        
        
        temp = temp.merge(weekly_mean_SPC(dates_non_str[i],dates_non_str[i]), on='GEOID', how='left')
        temp.rename(columns={'mean_spc':'SPC_'+ period}, inplace=True)


        temp = temp.merge(weekly_mean_FPC(dates_non_str[i],dates_non_str[i]), on='GEOID', how='left')
        temp.rename(columns={'mean_fpc':'FPC_'+ period}, inplace=True)


        
        
        i+=1
    
    times = ['T', 'T_1', 'T_2', 'T_3', 'T_4']
    
    j = 0
    for period in times:
        temp['DELTA_INC_RATE_' + period] = temp['inc_rate_' + time_periods[j]] - temp['inc_rate_' + time_periods[j+1]]
        temp['DELTA_REL_MOB_' + period] = temp['relative_' + time_periods[j]] - temp['relative_' + time_periods[j+1]]
        temp['DELTA_RATIO_MOB_' + period] = temp['ratio_' + time_periods[j]] - temp['ratio_' + time_periods[j+1]]
        temp['DELTA_SPC_' + period] = temp['SPC_' + time_periods[j]] - temp['SPC_' + time_periods[j+1]]
        temp['DELTA_FPC_' + period] = temp['FPC_' + time_periods[j]] - temp['FPC_' + time_periods[j+1]]
        
        temp['MEAN_INC_RATE_' + period] = covid_df[pd.date_range(start=dates[j+1], end=dates[j]).strftime('%Y-%m-%d')].mean(axis=1) / temp['POPULATION'] * 10000
        temp['MEAN_REL_MOB_' + period] = relative_df_smooth[pd.date_range(start=dates[j+1], end=dates[j]).strftime('%Y-%m-%d')].mean(axis=1)
        temp['MEAN_RATIO_MOB_' + period] = ratio_df_smooth[pd.date_range(start=dates[j+1], end=dates[j]).strftime('%Y-%m-%d')].mean(axis=1)

        
        # add Safegraph mobility features
        safegraph_data = safegraph_contiguous[safegraph_contiguous['end_date']==dates[j]][safegraph_metrics]
        temp = temp.merge(safegraph_data, left_on='GEOID', right_on='county_fips')
        rename_dict = dict()
        for col in safegraph_metrics[1:]:
            rename_dict[col] = col + '_' + period
            
        temp.rename(columns=rename_dict, inplace=True)
        
        
        # add MEAN_FPC
        temp = temp.merge(weekly_mean_FPC(dates_non_str[j], dates_non_str[j+1]), on='GEOID', how='left')
        temp.rename(columns={'mean_fpc':'MEAN_FPC_'+ period}, inplace=True)

        
        # add MEAN_SPC
        temp = temp.merge(weekly_mean_SPC(dates_non_str[j], dates_non_str[j+1]), on='GEOID', how='left')
        temp.rename(columns={'mean_spc':'MEAN_SPC_'+ period}, inplace=True)

        
        # add FB mobility slopes
        ratio_slope = linear_reg(weekly_fb_mobility(dates_non_str[j], dates_non_str[j+1], ratio_df_smooth))
        temp = temp.merge(ratio_slope, on='GEOID', how='left')
        temp.rename(columns={'slope':'SLOPE_RATIO_MOB_'+ period}, inplace=True)

        
        relative_slope = linear_reg(weekly_fb_mobility(dates_non_str[j], dates_non_str[j+1], relative_df_smooth))
        temp = temp.merge(relative_slope, on='GEOID', how='left')
        temp.rename(columns={'slope':'SLOPE_REL_MOB_'+ period}, inplace=True)
    

        
        
        # add temperature features
        # to update for the new dates, min and max temperature are used with one day offset
        adj_temp_date = (dates_non_str[j] + timedelta(days=1)).strftime('%Y-%m-%d')
        
        temp = temp.merge(max_temp[['GEOID', adj_temp_date]], on='GEOID', how='left')
        temp.rename(columns={adj_temp_date:'MAX_TEMP_'+ period}, inplace=True)
        
        temp = temp.merge(min_temp[['GEOID', adj_temp_date]], on='GEOID', how='left')
        temp.rename(columns={adj_temp_date:'MIN_TEMP_'+ period}, inplace=True)
        
        #temp = temp.merge(max_temp[['GEOID', dates[j]]], on='GEOID', how='left')
        #temp.rename(columns={dates[j]:'MAX_TEMP_'+ period}, inplace=True)
        
        #temp = temp.merge(min_temp[['GEOID', dates[j]]], on='GEOID', how='left')
        #temp.rename(columns={dates[j]:'MIN_TEMP_'+ period}, inplace=True)
        
        j += 2


    output_df = temp.copy()
    
    print('Feature generation for one week finished')
    
    return output_df

In [97]:
import warnings
warnings.filterwarnings('ignore')

In [98]:
# Rnu the function above for the end dates of 2020-11-29 to 2020-04-05, by 1 week difference

end_time = datetime(2020, 11, 28)
final_df = add_lagged_data(end_time)
#test = final_df.copy()

Feature generation for one week finished


In [101]:
end_time = datetime(2020, 11, 28)
while end_time > datetime(2020, 4, 5):
    end_time -= timedelta(weeks=1)
    final_df = pd.concat([final_df, add_lagged_data(end_time)])

Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generation for one week finished
Feature generati

In [102]:
final_df.shape

(108605, 308)

In [103]:
final_df.shape[0]/3103

35.0

### imputing NAs

In [104]:
# pd.set_option('max_rows', 200)
final_df.isna().sum().sum()

1382

In [121]:
pd.set_option('max_rows', 350)
final_df.isna().sum()

STATEFP                                               0
COUNTYFP                                              0
COUNTYNS                                              0
AFFGEOID                                              0
GEOID                                                 0
NAME                                                  0
LSAD                                                  0
ALAND                                                 0
AWATER                                                0
FIPS                                                  0
POPULATION                                            0
PCT_MALE                                              0
PCT_BLACK                                             0
PCT_HISPAN                                            0
PCT_COL_DE                                            0
PCT_RETAIL                                           35
PCT_AMIND                                             0
PCT_TRANSP                                      

In [105]:
data_to_save = final_df.copy()

In [106]:
na_columns = data_to_save.columns[data_to_save.isnull().any()]

In [107]:
# impute missing values by state average
for col in data_to_save[na_columns].columns:
    data_to_save[col] = data_to_save.groupby('STATEFP')[col].transform(lambda x: x.fillna(x.mean()))

In [108]:
data_to_save.isna().sum().sum()

0

In [109]:
times = ['T', 'T_1', 'T_2', 'T_3', 'T_4']

for period in times:
    data_to_save['LOG_DELTA_INC_RATE_' + period] = np.log(data_to_save['DELTA_INC_RATE_' + period] + 1)
    data_to_save['LOG_MEAN_INC_RATE_' + period] = np.log(data_to_save['MEAN_INC_RATE_' + period] + 1)
    data_to_save['LOG_DELTA_SPC_' + period] = np.log(data_to_save['DELTA_SPC_' + period] + 1)
    data_to_save['LOG_DELTA_FPC_' + period] = np.log(data_to_save['DELTA_FPC_' + period] + 1)

In [140]:
final_cols = [
'GEOID',
'NAME',
'State_Name',
'STATEFP', 
'COUNTYFP', 
'date_end_period',
'date_start_period',
'date_end_lag',
'date_start_lag',
'LOG_DELTA_INC_RATE_T',
'POP_DENSITY',
'PCT_MALE',
'PCT_BLACK',
'PCT_HISPAN', 
'PCT_AMIND',
'PCT_RURAL',
'PCT_COL_DE' ,
'PCT_TRUMP_',
'MED_HOS_IN',
]

In [141]:
additional_cols=[
'LOG_DELTA_INC_RATE_T_',
'DELTA_REL_MOB_T_',
'DELTA_RATIO_MOB_T_',
'DELTA_SPC_T_',
'DELTA_FPC_T_',
'LOG_MEAN_INC_RATE_T_',
'MEAN_REL_MOB_T_',
'MEAN_RATIO_MOB_T_',
'MEAN_FPC_T_',
'MEAN_SPC_T_',
'SLOPE_RATIO_MOB_T_',
'SLOPE_REL_MOB_T_',
'MAX_TEMP_T_',
'MIN_TEMP_T_',
'pct_completely_home_device_count_current_T_',
'pct_full_time_work_behavior_devices_current_T_',
'pct_part_time_work_behavior_devices_current_T_',
'pct_delivery_behavior_devices_current_T_',
'distance_traveled_from_home_current_T_',
'median_home_dwell_time_current_T_',
'pct_completely_home_device_count_baselined_T_',
'pct_full_time_work_behavior_devices_baselined_T_',
'pct_part_time_work_behavior_devices_baselined_T_',
'pct_delivery_behavior_devices_baselined_T_',
'distance_traveled_from_home_baselined_T_',
'median_home_dwell_time_baselined_T_',
'pct_completely_home_device_count_slope_T_',
'pct_full_time_work_behavior_devices_slope_T_',
'pct_part_time_work_behavior_devices_slope_T_',
'pct_delivery_behavior_devices_slope_T_',
'distance_traveled_from_home_slope_T_',
'median_home_dwell_time_slope_T_',
]

for i in range(1,5):
    for col in additional_cols:
        final_cols.append(col+str(i))
    

In [143]:
output = data_to_save[final_cols]

In [144]:
output.to_csv('./output/combined_df.csv')

In [145]:
# data_to_save.drop('geometry', axis=1).to_csv('./output/final_all_features_updated_dates.csv')

In [267]:
# data_to_save.to_file('./output/final_all_features.gojson', driver='GeoJSON')

In [146]:
# output = output.merge(data[['GEOID','POPULATION']], how='left', on='GEOID')

In [147]:
# backup_output = output.copy()

### Creating dataframes for 14-day and 21-day predictions

In [173]:
# Main function
# the input to the funtion is the end date for which we want to add data

from datetime import datetime, timedelta


def add_lagged_y(date):
    global output
    
    T_end = date
    T_start = T_end - timedelta(days=6)
    
    T_14 =  T_end + timedelta(days=7)
    T_21 =  T_end + timedelta(days=14)
    T_28 =  T_end + timedelta(days=21)
    


    dates = [T_end.strftime('%Y-%m-%d'), T_start.strftime('%Y-%m-%d'),
             T_14.strftime('%Y-%m-%d'), T_21.strftime('%Y-%m-%d'), T_28.strftime('%Y-%m-%d')]
    
    dates_non_str = [T_end, T_start, T_14, T_21, T_28]
    periods = ['T_end', 'T_start', 'T_14', 'T_21', 'T_28']
    
    temp = output.loc[(output.date_end_period==dates[0]) & (output.date_start_period==dates[1])].copy()
    
    temp['target_date_2wk'] = T_14.strftime('%Y-%m-%d')
    temp['target_date_3wk'] = T_21.strftime('%Y-%m-%d')
    temp['target_date_4wk'] = T_28.strftime('%Y-%m-%d')
        
    temp = temp.merge(covid_df_contiguous[['GEOID',*dates]], on='GEOID', how='left')


    for period, date in zip(periods, dates):
        temp['inc_rate_' + period] = temp[date] / temp['POPULATION'] * 10000


    for period in periods[-3:]:
        temp['DELTA_INC_RATE_' + period] = temp['inc_rate_' + period] - temp['inc_rate_T_start']
        temp['LOG_DELTA_INC_RATE_' + period] = np.log(temp['DELTA_INC_RATE_' + period] + 1)
    
    
    cols = ['target_date_2wk','LOG_DELTA_INC_RATE_T_14', 
            'target_date_3wk','LOG_DELTA_INC_RATE_T_21',
            'target_date_4wk','LOG_DELTA_INC_RATE_T_28' ]
    
    
    return temp[[*output.columns,'target_date_2wk','LOG_DELTA_INC_RATE_T_14', 
            'target_date_3wk','LOG_DELTA_INC_RATE_T_21',
            'target_date_4wk','LOG_DELTA_INC_RATE_T_28']]

In [168]:
initial_time = datetime(2020, 11, 28)
lagged_y_df = add_lagged_y(initial_time)

In [174]:
end_time =  datetime(2020, 11, 21)
while end_time > datetime(2020, 3, 29):
    #add_lagged_y(end_time)
    lagged_y_df = pd.concat([lagged_y_df, add_lagged_y(end_time)])   
    end_time -= timedelta(weeks=1)

In [177]:
lagged_y_df.isna().sum().sum()

0

In [183]:
lagged_y_df.shape[0]/3103

35.0

In [180]:
lagged_y_df.to_csv('./output/combined_df_lagged_y.csv')